In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

timeout = 25

In [2]:
driver = webdriver.Chrome()

scrape = 3

In [3]:
url = f"https://capitalcityonlineauction.com/Public/Auction"
driver.get(url)
element_present = EC.presence_of_element_located((By.XPATH, r"/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div[2]/div/div[3]/div[1]"))
WebDriverWait(driver, timeout).until(element_present)
#List to hold item info
templist = []
#Get Links on Homepage
links = []
#Will add automatic ceiling for i
i = 1
#Get all links from home page ----needs a +1
while i < scrape:
    target = driver.find_element_by_xpath(f'//*[@id="AuctionList"]/div[1]/div[{i}]/div/div[2]/div[3]/a').get_attribute('onclick')
    length = len(target)
    site = "https://capitalcityonlineauction.com"+target[21:(length-2)]
    links.append(site)
    i+=1

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=104.0.5112.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A35FD3+2187219]
	Ordinal0 [0x009CE6D1+1763025]
	Ordinal0 [0x008E3E78+802424]
	Ordinal0 [0x008D9AC4+760516]
	Ordinal0 [0x008DAB02+764674]
	Ordinal0 [0x008D4379+738169]
	Ordinal0 [0x008E51D0+807376]
	Ordinal0 [0x0093C7A2+1165218]
	Ordinal0 [0x0092C9A6+1100198]
	Ordinal0 [0x00906F80+946048]
	Ordinal0 [0x00907E76+949878]
	GetHandleVerifier [0x00CD90C2+2721218]
	GetHandleVerifier [0x00CCAAF0+2662384]
	GetHandleVerifier [0x00AC137A+526458]
	GetHandleVerifier [0x00AC0416+522518]
	Ordinal0 [0x009D4EAB+1789611]
	Ordinal0 [0x009D97A8+1808296]
	Ordinal0 [0x009D9895+1808533]
	Ordinal0 [0x009E26C1+1844929]
	BaseThreadInitThunk [0x74EEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77067A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77067A6E+238]


In [ ]:
j=0

In [ ]:
#open pages loop
while j < (scrape-1):
    auction = links[j]
    driver.get(auction)
    #Count Page numbers in Auction
    pages = int(driver.find_element_by_xpath('//*[@id="page-wrapper"]/div[2]/div[1]/div[1]/div[4]/div/div[5]/div/div[1]/div[2]/div[5]/span[2]').text)
    #Loop through pages
    k=0
    while k < pages:
        #Select Page [k], skip select if first page
        if k > 0 :
            driver.find_element_by_xpath(f'//*[@id="contentPager"]/nav/ul/li[{(k+3)}]/a').click()
        else:
            k=0
        #Count items on current page by splitting x - y of z items
        page_items = driver.find_element_by_xpath('//*[@id="contentPager"]/div').text
        #Not needed, helps navigate
        print(page_items)
        #Splits string by spaces
        str = page_items.split()
        #first item number of page
        first = int(str[0])
        #last item number of page
        total = int(str[2])
        #Get total items on page
        items = total - first
        #loop through items
        count=0 
        while count < items+1:
            #seems to have one of two xpaths, selects whichver is present to get item url 
            #Broke here ---- if driver = None? Element not present?
            try:
                driver.find_element_by_xpath(f'//*[@id="divAuctionItem_0"]/div[1]/h4[2]/a')
                connect = driver.find_element_by_xpath(f'//*[@id="divAuctionItem_{count}"]/div[1]/h4[2]/a').get_attribute('href')
            except:
                try:
                    connect = driver.find_element_by_xpath(f'//*[@id="AuctionItemsList"]/div[4]/div/div[{count+1}]/div[2]/div/div[1]/h4[2]/a').get_attribute('href')
                except:
                    connect = driver.find_element_by_xpath(f'//*[@id="carouselExampleControls_{count}"]/div/a').get_attribute('href')
            print(connect)
            #Auction Location Text
            location = driver.find_element_by_xpath('//*[@id="page-wrapper"]/div[2]/div[1]/div[1]/div[4]/div/div[2]/div[2]/div[1]/span').text
            #Auction End Time
            end = driver.find_element_by_xpath('//*[@id="page-wrapper"]/div[2]/div[1]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div/div[2]/p[2]').text
            #Item Lot Number --- Appears to be a persistant issue with different xpaths
            try:
                driver.find_element_by_xpath(f'//*[@id="divAuctionItem_0"]/div[1]/h4[1]/a')
                lot = driver.find_element_by_xpath(f'//*[@id="divAuctionItem_{count}"]/div[1]/h4[1]/a/span').text
            except:
                lot = driver.find_element_by_xpath(f'//*[@id="AuctionItemsList"]/div[4]/div/div[{count+1}]/div[2]/div/div[1]/h4[1]/a/span').text
            #Item Decriptions
            try:
                driver.find_element_by_xpath(f'//*[@id="divAuctionItem_0"]/div[1]/h4[2]/a')
                desc = driver.find_element_by_xpath(f'//*[@id="divAuctionItem_{count}"]/div[1]/h4[2]/a').text
            except:
                try:
                    desc = driver.find_element_by_xpath(f'//*[@id="AuctionItemsList"]/div[4]/div/div[{count+1}]/div[2]/div/div[1]/h4[2]/a').text
                except:
                    desc = "None"
            #Item Sku
            try:
                driver.find_element_by_xpath(f'//*[@id="divAuctionItem_0"]/div[1]/p')
                sku = driver.find_element_by_xpath(f'//*[@id="divAuctionItem_{count}"]/div[1]/p').text
            except:
                sku = driver.find_element_by_xpath(f'//*[@id="AuctionItemsList"]/div[4]/div/div[{count+1}]/div[2]/div/div[1]/p').text
            #Item Bullets, MSRP most important here but likely will remove
            try:
                driver.find_element_by_xpath(f'//*[@id="divAuctionItem_0"]/div[2]/div/div')
                bul = driver.find_element_by_xpath(f'//*[@id="divAuctionItem_{count}"]/div[2]/div/div').text
            except:
                bul = driver.find_element_by_xpath(f'//*[@id="AuctionItemsList"]/div[4]/div/div[{count+1}]/div[2]/div/div[1]/div[2]').text
            #Current Bid
            bid = driver.find_element_by_xpath(f'//*[@id="CurrentBidAmount_{count}"]').text
            #Next Min Bid To be added. Everything looks good in pop and manually navigated to window...not essential
            #bidnext = driver.find_element_by_xpath(f'//*[@id="divAuctionItem_{count}"]/div[7]/div').text
            #print(bidnext)
            #Puts in df table
            Table_dict={'URL': connect, 'Location': location, 'End Time': end, 'Lot No.': lot, 'Description': desc, 'Sku': sku, 'Bullets': bul, 'Current Bid': bid}
            templist.append(Table_dict)
            df = pd.DataFrame(templist)
            count+=1
        k+=1
    j+=1
#export to csv
df.to_csv(f'C:\\Users\\Steven\\Desktop\\ItemsList.csv') 
    

In [ ]:
print("Complete")